<a href="https://colab.research.google.com/github/MINHDUC-md/minhduc/blob/main/MinhDuc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bước 1: Import thư viện

In [ ]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ
from sklearn.preprocessing import MinMaxScaler #chuẩn hóa dữ liệu
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất
from tensorflow.keras.models import load_model #tải mô hình

#các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM #học phụ thuộc
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra

#kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_absolute_error #đo sai số tuyệt đối trung bình
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình

# Bước 2: Đọc dữ liệu

In [ ]:
#tạo quyền truy cập với google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# đọc dữ liệu từ file csv
df = pd.read_csv('/content/drive/MyDrive/MyDrive/Dữ-liệu-Lịch-sử-VNM.csv')

# Xóa hai dòng "KL" và "Thay đổi %" từ DataFrame dataSet
df = df.drop(columns=["KL", "% Thay đổi"])

# Hiển thị lại DataFrame sau khi xóa
print(df)

# Bước 3: Mô tả dữ liệu

In [ ]:
#định dạng cấu trúc thời gian
df["Ngày"] = pd.to_datetime(df.Ngày,format="%d/%m/%Y")

In [ ]:
#kích thước dữ liệu
df.shape

In [ ]:
#dữ liệu 5 dòng đầu
df.head()

In [ ]:
#xác định kiểu dữ liệu
df.info()

In [ ]:
#mô tả bộ dữ liệu
df.describe()

# Bước 4: Tiền xử lý dữ liệu

In [ ]:
from matplotlib.dates import MonthLocator, DateFormatter
import matplotlib.pyplot as plt
import pandas as pd

# Chuyển đổi cột "Ngày" sang dạng datetime
df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')

# Sắp xếp lại dữ liệu theo thứ tự thời gian
df = df.sort_values(by='Ngày')

# Kiểm tra kiểu dữ liệu của các cột và chuyển đổi nếu cần
for column in ['Lần cuối', 'Mở', 'Cao', 'Thấp']:
    if column in df.columns and df[column].dtype == 'object':
        df[column] = df[column].str.replace(',', '').astype(float)

# Lấy thông tin tháng từ cột "Ngày"
df['Tháng'] = df['Ngày'].dt.month

# Tạo đồ thị giá đóng cửa qua các tháng
plt.figure(figsize=(10, 5))
plt.plot(df['Ngày'], df['Lần cuối'], label='Giá đóng cửa', color='red')
plt.xlabel('Tháng')
plt.ylabel('Giá đóng cửa')
plt.title('Biểu đồ giá đóng cửa theo tháng')

# Định dạng trục x theo từng tháng
months = MonthLocator()  # Đặt tick chính là từng tháng
monthsFmt = DateFormatter('%b %Y')  # Hiển thị định dạng tháng-năm

plt.gca().xaxis.set_major_locator(months)
plt.gca().xaxis.set_major_formatter(monthsFmt)

plt.legend(loc='best')
plt.tight_layout()
plt.show()


In [ ]:
# Tạo DataFrame với cột 'Ngày' và 'Lần cuối'
df1 = pd.DataFrame(df, columns=['Ngày', 'Lần cuối'])

# Đặt cột 'Ngày' làm index
df1.index = df1['Ngày']

# Bỏ cột 'Ngày' vì nó đã được đặt làm index
df1.drop('Ngày', axis=1, inplace=True)

# Hiển thị DataFrame sau khi chỉnh sửa
df1.head()


In [ ]:
#chia tập dữ liệu
data = df1.values
train_data = data[:1500]
test_data = data[1500:]

In [ ]:
#chuẩn hóa dữ liệu
sc = MinMaxScaler(feature_range=(0,1))
sc_train = sc.fit_transform(data)

In [ ]:
data

In [ ]:
#tạo vòng lặp các giá trị
x_train,y_train=[],[]
for i in range(50,len(train_data)):
  x_train.append(sc_train[i-50:i,0]) #lấy 50 giá đóng cửa liên tục
  y_train.append(sc_train[i,0]) #lấy ra giá đóng cửa ngày hôm sau

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
#xếp dữ liệu thành 1 mảng 2 chiều
x_train = np.array(x_train)
y_train = np.array(y_train)

#xếp lại dữ liệu thành mảng 1 chiều
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train = np.reshape(y_train,(y_train.shape[0],1))

# Bước 5: Xây dựng và huấn luyện mô hình

In [ ]:
#xây dựng mô hình
model = Sequential() #tạo lớp mạng cho dữ liệu đầu vào
#2 lớp LSTM
model.add(LSTM(units=128,input_shape=(x_train.shape[1],1),return_sequences=True))
model.add(LSTM(units=64))
model.add(Dropout(0.5)) #loại bỏ 1 số đơn vị tránh học tủ (overfitting)
model.add(Dense(1)) #output đầu ra 1 chiều
#đo sai số tuyệt đối trung bình có sử dụng trình tối ưu hóa adam
model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
#huấn luyện mô hình
save_model = "/content/drive/MyDrive/MyDrive/save_model.keras"
best_model = ModelCheckpoint(save_model,monitor='loss',verbose=2,save_best_only=True,mode='auto')
model.fit(x_train,y_train,epochs=100,batch_size=50,verbose=2,callbacks=[best_model])

In [ ]:
#dữ liệu train
y_train = sc.inverse_transform(y_train) #giá thực
final_model = load_model("/content/drive/MyDrive/MyDrive/save_model.keras")
y_train_predict = final_model.predict(x_train) #dự đoán giá đóng cửa trên tập đã train
y_train_predict = sc.inverse_transform(y_train_predict) #giá dự đoán

# Bước 6: Tái sử dụng mô hình

In [ ]:
# Xử lý dữ liệu test
# Giả sử bạn chỉ muốn sử dụng cột đầu tiên trong dữ liệu mới (có thể thay đổi chỉ số cột nếu cần)
test = df1[len(train_data) - 106:].values  # Lấy 106 hàng từ df1
test = test[:, 0].reshape(-1, 1)  # Chỉ lấy cột đầu tiên và định hình lại

# Chuẩn hóa dữ liệu
sc_test = sc.transform(test)

# Tạo x_test
x_test = []
for i in range(50, sc_test.shape[0]):  # Bắt đầu từ chỉ số 50
    x_test.append(sc_test[i - 50:i, 0])

# Chuyển đổi x_test thành mảng NumPy
x_test = np.array(x_test)

# Đảm bảo kích thước phù hợp cho LSTM
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Dữ liệu test (giá thực)
# Lưu ý: Bạn cần đảm bảo rằng y_test cũng có kích thước phù hợp
y_test = data[1500:1500 + len(x_test)]  # Lấy giá thực tương ứng với x_test
y_test = y_test[:, 0]  # Giả sử bạn lấy cột đầu tiên, có thể thay đổi chỉ số cột

# Dự đoán
y_test_predict = final_model.predict(x_test)
y_test_predict = sc.inverse_transform(y_test_predict)  # Giá dự đoán

# Kiểm tra kích thước y_test và y_test_predict
print("Kích thước y_test:", y_test.shape)
print("Kích thước y_test_predict:", y_test_predict.shape)


# Độ chính xác của mô hình

In [ ]:
import matplotlib.pyplot as plt

# Giả sử df1 có ít nhất 160 hàng và có cấu trúc phù hợp
# Lấy dữ liệu huấn luyện và kiểm tra
train_data1 = df1[:50]  # Lấy 50 hàng đầu tiên cho dữ liệu huấn luyện
test_data1 = df1[50:]    # Lấy 110 hàng còn lại cho dữ liệu kiểm tra

# Tạo biểu đồ
plt.figure(figsize=(24, 8))
plt.plot(df1, label='Giá thực tế', color='red')  # Đường giá thực

# Thêm dự đoán cho dữ liệu huấn luyện
train_data1['Dự đoán'] = y_train_predict[:50]  # Chỉ sử dụng 50 dự đoán đầu tiên
plt.plot(train_data1.index, train_data1['Dự đoán'], label='Giá dự đoán train', color='green')  # Đường giá dự báo train

# Thêm dự đoán cho dữ liệu kiểm tra
test_data1['Dự đoán'] = y_test_predict  # Sử dụng tất cả dự đoán cho dữ liệu kiểm tra
plt.plot(test_data1.index, test_data1['Dự đoán'], label='Giá dự đoán test', color='blue')  # Đường giá dự báo test

# Thiết lập tiêu đề và nhãn
plt.title('So sánh giá dự báo và giá thực tế')  # Đặt tên biểu đồ
plt.xlabel('Thời gian')  # Đặt tên trục x
plt.ylabel('Giá đóng cửa (VNĐ)')  # Đặt tên trục y
plt.legend()  # Chú thích
plt.show()  # Hiển thị biểu đồ


In [ ]:
#r2
print('Độ phù hợp tập train:',r2_score(y_train,y_train_predict))
#mae
print('Sai số tuyệt đối trung bình trên tập train (VNĐ):',mean_absolute_error(y_train,y_train_predict))
#mae
print('Phần trăm sai số tuyệt đối trung bình tập train:',mean_absolute_percentage_error(y_train,y_train_predict))

In [ ]:
train_data1

In [ ]:
#r2
print('Độ phù hợp tập test:',r2_score(y_test,y_test_predict))
#mae
print('Sai số tuyệt đối trung bình trên tập test (VNĐ):',mean_absolute_error(y_test,y_test_predict))
#mae
print('Phần trăm sai số tuyệt đối trung bình tập test:',mean_absolute_percentage_error(y_test,y_test_predict))

In [ ]:
test_data1

# Thử dự đoán giá tương lai và so sánh với giá ngày cuối

In [ ]:
# Lấy ngày kế tiếp sau ngày cuối cùng trong tập dữ liệu để dự đoán
next_date = df['Ngày'].iloc[-1] + pd.Timedelta(days=1)

# Chuyển đổi ngày kế tiếp sang dạng datetime
next_date = pd.to_datetime(next_date)

# Lấy giá trị của ngày cuối cùng trong tập dữ liệu
next_closing_price = np.array([df['Lần cuối'].iloc[-1]])  # Lấy giá trị đóng cửa của ngày cuối cùng

# Chuẩn hóa giá trị của ngày cuối cùng
next_closing_price_normalized = sc.transform(next_closing_price.reshape(-1, 1))  # Chuyển thành mảng 2D

# Tạo dự đoán cho ngày kế tiếp bằng mô hình đã huấn luyện
x_next = np.array([sc_train[-50:, 0]])  # Lấy 50 giá đóng cửa gần nhất
x_next = np.reshape(x_next, (x_next.shape[0], x_next.shape[1], 1))
y_next_predict = final_model.predict(x_next)
y_next_predict = sc.inverse_transform(y_next_predict)

# Thêm dữ liệu dự đoán của ngày kế tiếp vào DataFrame
df_next = pd.DataFrame({'Ngày': [next_date], 'Lần cuối': [y_next_predict[0][0]]})
df1 = pd.concat([df1, df_next])

# Vẽ biểu đồ mới với dự đoán cho ngày kế tiếp
plt.figure(figsize=(15, 5))
plt.plot(df1['Ngày'], df1['Lần cuối'], label='Giá thực tế', color='red')
plt.plot(train_data1.index, train_data1['Dự đoán'], label='Giá dự đoán train', color='green')
plt.plot(test_data1.index, test_data1['Dự đoán'], label='Giá dự đoán test', color='blue')
plt.scatter([next_date], [y_next_predict[0][0]], color='orange', label='Dự đoán ngày kế tiếp')
plt.xlabel('Thời gian')
plt.ylabel('Giá đóng cửa (VNĐ)')
plt.title('So sánh giá dự báo và giá thực tế')
plt.legend()
plt.show()


In [ ]:
# Lấy giá trị của ngày cuối cùng trong tập dữ liệu
actual_closing_price = df['Lần cuối'].iloc[-1]

# Tạo DataFrame so sánh giá dự đoán với giá ngày cuối trong tập dữ liệu
comparison_df = pd.DataFrame({'Ngày': [next_date], 'Giá dự đoán': [y_next_predict[0][0]], 'Giá ngày trước': [actual_closing_price]})

# In ra bảng so sánh
print(comparison_df)